In [2]:
import sys
sys.path.append('../../')
import cvxpy as cvx
import numpy as np
from QuantumTomography import RandomUnitary, Process2Choi

In [3]:
dim = 4
U = np.zeros((dim,dim))
U[0,0] = 1
V = np.zeros((dim,dim))
V[-1,-1] = 1
Yu = Process2Choi( np.kron(U,U.conj()) )
Yv = Process2Choi( np.kron(V,V.conj()) )


In [4]:
def DiamondNorm( Yu, Yv, type='choi' ):

    if type == 'vec':
        Yu = Process2Choi( Yu )
        Yv = Process2Choi( Yv )

    delta_choi = Yu - Yv

    # Density matrix must be Hermitian, positive semidefinite, trace 1
    rho = cvx.Variable([dim, dim], complex=True)
    constraints = [rho == rho.H]
    constraints += [rho >> 0]
    constraints += [cvx.trace(rho) == 1]

    # W must be Hermitian, positive semidefinite
    W = cvx.Variable([dim ** 2, dim ** 2], complex=True)
    constraints += [W == W.H]
    constraints += [W >> 0]

    constraints += [(W - cvx.kron(np.eye(dim), rho)) << 0]

    J = cvx.Parameter([dim ** 2, dim ** 2], complex=True)
    objective = cvx.Maximize(cvx.real(cvx.trace(J.H @ W)))

    prob = cvx.Problem(objective, constraints)

    J.value = delta_choi
    prob.solve()

    dnorm = prob.value * 2

    # Diamond norm is between 0 and 2. Correct for floating point errors
    dnorm = min(2, dnorm)
    dnorm = max(0, dnorm)

    return dnorm

In [7]:
dnorm = DiamondNorm( Yu, Yv )
dnorm

1.9999773670516954

In [8]:
np.linalg.norm( Yu-Yv, 'fro' )

1.4142135623730951

In [10]:
Yu.shape[0] 

16